In [1]:
import pandapower as pp
import pypower.api as pyp
# import numba

In [2]:
ppc = pyp.case57()
# pp.diagnostic(net)

In [3]:
import pandapower.networks as pn

net2 = pn.case57()
ppc2 = pp.converter.to_ppc(net2, init="flat")


In [6]:
net2.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,max_p_mw,min_p_mw
0,None,1,0.0,1.010,NaN,-17.0,50.0,1.0,False,True,0.0,None,True,100.0,0.0
1,None,2,40.0,0.985,NaN,-10.0,60.0,1.0,False,True,0.0,None,True,140.0,0.0
2,None,5,0.0,0.980,NaN,-8.0,25.0,1.0,False,True,0.0,None,True,100.0,0.0
3,None,7,450.0,1.005,NaN,-140.0,200.0,1.0,False,True,0.0,None,True,550.0,0.0
4,None,8,0.0,0.980,NaN,-3.0,9.0,1.0,False,True,0.0,None,True,100.0,0.0
5,None,11,310.0,1.015,NaN,-150.0,155.0,1.0,False,True,0.0,None,True,410.0,0.0


In [9]:
net2.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,min_vm_pu
0,1,115.0,b,1.0,True,1.06,0.94
1,2,115.0,b,1.0,True,1.06,0.94
2,3,115.0,b,1.0,True,1.06,0.94
3,4,115.0,b,1.0,True,1.06,0.94
4,5,115.0,b,1.0,True,1.06,0.94
5,6,115.0,b,1.0,True,1.06,0.94
6,7,115.0,b,1.0,True,1.06,0.94
7,8,115.0,b,1.0,True,1.06,0.94
8,9,115.0,b,1.0,True,1.06,0.94
9,10,115.0,b,1.0,True,1.06,0.94


In [54]:
net = pp.converter.from_ppc(ppc2)
pp.runpp(net, numba=False)

In [52]:
ppc2["bus"]

array([[  0.   ,   3.   ,  55.   ,  17.   ,   0.   ,   0.   ,   1.   ,
          1.04 ,   0.   , 115.   ,   1.   ,   1.04 ,   1.04 ,   0.   ,
          0.   ,   0.   ],
       [  1.   ,   2.   ,   3.   ,  88.   ,   0.   ,   0.   ,   1.   ,
          1.01 ,   0.   , 115.   ,   1.   ,   1.06 ,   0.94 ,   0.   ,
          0.   ,   0.   ],
       [  2.   ,   2.   ,  41.   ,  21.   ,   0.   ,   0.   ,   1.   ,
          0.985,   0.   , 115.   ,   1.   ,   1.06 ,   0.94 ,   0.   ,
          0.   ,   0.   ],
       [  3.   ,   1.   ,   0.   ,   0.   ,   0.   ,   0.   ,   1.   ,
          1.   ,   0.   , 115.   ,   1.   ,   1.06 ,   0.94 ,   0.   ,
          0.   ,   0.   ],
       [  4.   ,   1.   ,  13.   ,   4.   ,   0.   ,   0.   ,   1.   ,
          1.   ,   0.   , 115.   ,   1.   ,   1.06 ,   0.94 ,   0.   ,
          0.   ,   0.   ],
       [  5.   ,   2.   ,  75.   ,   2.   ,   0.   ,   0.   ,   1.   ,
          0.98 ,   0.   , 115.   ,   1.   ,   1.06 ,   0.94 ,   0.   ,
          0. 

In [24]:

net1.bus.loc[:, "vn_kv"] = net2.bus.loc[:, "vn_kv"]

In [25]:
pp.runpp(net1, numba=False)

/home/jxxiong/.conda/envs/scopf/lib/python3.9/site-packages/pandapower/build_branch.py:436: RuntimeWarning: invalid value encountered in divide
  trafo_shift[tap_complex] += (arctan(direction * du * sin(tap_angles) /
/home/jxxiong/.conda/envs/scopf/lib/python3.9/site-packages/pandapower/build_branch.py:576: RuntimeWarning: invalid value encountered in divide
  tap_rat = vn_hv_kv / vn_lv_kv
/home/jxxiong/.conda/envs/scopf/lib/python3.9/site-packages/pandapower/build_branch.py:373: RuntimeWarning: invalid value encountered in divide
  b_real = pfe / vnl_squared * baseR
/home/jxxiong/.conda/envs/scopf/lib/python3.9/site-packages/pandapower/build_branch.py:380: RuntimeWarning: invalid value encountered in divide
  b_img = np.sqrt(b_img) * baseR / vnl_squared
/home/jxxiong/.conda/envs/scopf/lib/python3.9/site-packages/pandapower/build_branch.py:382: RuntimeWarning: invalid value encountered in divide
  return y / np.square(vn_trafo_lv / vn_lv_kv) * parallel
/home/jxxiong/.conda/envs/scopf/l

LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

In [ ]:
net.res_bus

In [ ]:
om = pypower.opf.opf_setup(ppc, ppopt)
om.add_constraints('P', csr_array(np.eye(5)), np.ones((5, 1)), np.ones((5, 1)), ['Pg'])

In [ ]:
om

In [ ]:
pp.opf_execute(om, ppopt)

In [ ]:
net